In [17]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [18]:
def extract_features(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (128,128))

  _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  th_inv = cv2.bitwise_not(th)

  area = np.sum(th_inv == 255)

  contours, _ = cv2.findContours(th_inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  perimeter = cv2.arcLength(contours[0], True) if len(contours) > 0 else 0

  mean_intensity = np.mean(gray)
  std_intensity = np.std(gray)

  hist = cv2.calcHist([gray], [0], None, [16], [0,256])
  hist = cv2.normalize(hist, hist).flatten()

  features = [
    area,
    perimeter,
    mean_intensity,
    std_intensity
  ]

  features.extend(hist)

  return np.array(features)


In [19]:
data = []
labels = []

base = "dataset/train"
classes = ["50", "100", "200", "500", "1000"]

for cls in classes:
  folder = os.path.join(base, cls)
  for file in os.listdir(folder):
    path = os.path.join(folder, file)
    img = cv2.imread(path)
    if img is None:
      continue

    feat = extract_features(img)
    data.append(feat)
    labels.append(cls)

data = np.array(data)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
  data, labels, test_size=0.2, random_state=42, stratify=labels
)

model = RandomForestClassifier(n_estimators=300, random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_test)

print(classification_report(y_test, pred))


              precision    recall  f1-score   support

         100       0.80      0.67      0.73         6
        1000       0.50      0.80      0.62         5
         200       1.00      0.67      0.80         6
          50       0.80      0.67      0.73         6
         500       0.83      1.00      0.91         5

    accuracy                           0.75        28
   macro avg       0.79      0.76      0.76        28
weighted avg       0.80      0.75      0.76        28



In [20]:
import joblib
joblib.dump(model, "coin_rf.pkl")


['coin_rf.pkl']

In [21]:
correct = 0
total = 0

base = "dataset/test"

for cls in classes:
  folder = os.path.join(base, cls)
  for file in os.listdir(folder):
    img = cv2.imread(os.path.join(folder, file))
    if img is None:
      continue

    feat = extract_features(img).reshape(1, -1)
    pred = model.predict(feat)[0]

    print(file, "=> pred:", pred, " | true:", cls)

    total += 1
    correct += (pred == cls)

print("\nAccuracy test folder:", correct / total)


IMG_50_000.jpg => pred: 50  | true: 50
IMG_50_001.jpg => pred: 50  | true: 50
IMG_50_002.jpg => pred: 50  | true: 50
IMG_50_003.jpg => pred: 50  | true: 50
IMG_100_000.jpg => pred: 500  | true: 100
IMG_100_001.jpg => pred: 100  | true: 100
IMG_100_002.jpg => pred: 100  | true: 100
IMG_100_003.jpg => pred: 100  | true: 100
IMG_200_000.jpg => pred: 200  | true: 200
IMG_200_001.jpg => pred: 1000  | true: 200
IMG_200_002.jpg => pred: 200  | true: 200
IMG_200_003.jpg => pred: 200  | true: 200
IMG_500_000.jpg => pred: 500  | true: 500
IMG_500_001.jpg => pred: 500  | true: 500
IMG_500_002.jpg => pred: 500  | true: 500
IMG_500_003.jpg => pred: 50  | true: 500
IMG_1000_000.jpg => pred: 1000  | true: 1000
IMG_1000_001.jpg => pred: 1000  | true: 1000
IMG_1000_002.jpg => pred: 1000  | true: 1000
IMG_1000_003.jpg => pred: 1000  | true: 1000

Accuracy test folder: 0.85


In [22]:
import gradio as gr
import cv2
import numpy as np
import joblib
from skimage.feature import hog
import os

# -----------------------------
# 1. FUNGSI FEATURE EXTRACTION
# -----------------------------
def extract_features(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (128,128))

  # Otsu
  _, th = cv2.threshold(gray, 0, 255,
                        cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  th_inv = cv2.bitwise_not(th)

  area = np.sum(th_inv == 255)

  contours, _ = cv2.findContours(th_inv,
                                 cv2.RETR_EXTERNAL,
                                 cv2.CHAIN_APPROX_SIMPLE)
  perimeter = cv2.arcLength(contours[0], True) if len(contours) > 0 else 0

  mean_intensity = np.mean(gray)
  std_intensity = np.std(gray)

  hist = cv2.calcHist([gray], [0], None, [16], [0,256])
  hist = cv2.normalize(hist, hist).flatten()

  features = [
    area,
    perimeter,
    mean_intensity,
    std_intensity
  ]
  features.extend(hist)

  return np.array(features)


# -----------------------------
# 2. DETEKSI KOIN
# -----------------------------
def detect_coins(image):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  _, th = cv2.threshold(gray, 0, 255,
                        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
  morph = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=2)

  cnts, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  circles = []
  for c in cnts:
    area = cv2.contourArea(c)
    if area < 2500:
      continue

    (x, y), r = cv2.minEnclosingCircle(c)
    circles.append((int(x), int(y), int(r)))

  return circles


# -----------------------------
# 3. DETEKSI DAN LABEL
# -----------------------------
model = joblib.load("coin_rf.pkl")

def detect_and_label(image):
  if image is None:
    return None

  img = image.copy()

  circles = detect_coins(img)

  for x, y, r in circles:
    x1 = max(0, x - r)
    y1 = max(0, y - r)
    x2 = min(img.shape[1], x + r)
    y2 = min(img.shape[0], y + r)

    coin = img[y1:y2, x1:x2]
    if coin.size == 0:
      continue

    feat = extract_features(coin).reshape(1, -1)
    pred = model.predict(feat)[0]

    cv2.circle(img, (x,y), r, (0,255,0), 3)
    cv2.putText(img, pred, (x - r, y - r),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)

  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img


# -----------------------------
# 4. GRADIO UI SIDE-BY-SIDE
# -----------------------------
with gr.Blocks() as demo:
  gr.Markdown("## Deteksi & Klasifikasi Koin Rupiah — RandomForest")

  with gr.Row():
    with gr.Column():
      inp = gr.Image(type="numpy", label="Upload Gambar")

      btn = gr.Button("Deteksi Koin")

    with gr.Column():
      out = gr.Image(type="numpy", label="Hasil Deteksi")

  btn.click(detect_and_label, inputs=inp, outputs=out)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
